In [1]:
import glob
from utils.analysis import Extrapolation

In [2]:
TAG = "pku"
babies = glob.glob(f"../analysis/studies/vbswh/output_{TAG}/Run2/*.root")
data_babies = [baby for baby in babies if "data" in baby]
sig_babies = [baby for baby in babies if "VBSWH_mkW" in baby]
bkg_babies = list(set(babies) - set(data_babies + sig_babies))
print(data_babies)
print(sig_babies)
bkg_babies

['../analysis/studies/vbswh/output_pku/Run2/data.root']
['../analysis/studies/vbswh/output_pku/Run2/VBSWH_mkW.root']


['../analysis/studies/vbswh/output_pku/Run2/WJets.root',
 '../analysis/studies/vbswh/output_pku/Run2/EWKWLep.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTX.root',
 '../analysis/studies/vbswh/output_pku/Run2/Bosons.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTbar2L.root',
 '../analysis/studies/vbswh/output_pku/Run2/SingleTop.root',
 '../analysis/studies/vbswh/output_pku/Run2/VH.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTbar1L.root']

In [3]:
vbswh = Extrapolation(
    sig_root_files=sig_babies,
    bkg_root_files=bkg_babies,
    data_root_files=data_babies,
    ttree_name="tree",
    weight_columns=[
        "xsec_sf", "lep_id_sf", "elec_reco_sf", "muon_iso_sf", 
        "btag_sf", "pu_sf", "prefire_sf", "trig_sf"
    ],
    plots_dir=f"/home/users/jguiang/public_html/onelep_plots/{TAG}/val"
)
vbswh.df["presel_noVBS_noBVeto"] = vbswh.df.eval(
    "hbbjet_score > 0.3"
)
vbswh.df["presel_noVBS"] = vbswh.df.eval(
    "passes_bveto and hbbjet_score > 0.3"
)
vbswh.df["presel_noDetaJJ"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and hbbjet_score > 0.3"
)
vbswh.df["presel"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and abs(deta_jj) > 3 and hbbjet_score > 0.3"
)

Loading data babies: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.19s/it]


In [4]:
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9",
#     "presel_noDetaJJ and M_jj > 600 and LT > 400 and ST > 900 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
#     show_raw=True,
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,190.3782778440434,3.4755115093738826,10.085911672694348,1.3074765499057894,148,12.165525060596439
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,247.28131394309912,5.902043357451355,0.8942036641677474,0.43743234037268064,200,14.142135623730951
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,186.08351580171507,4.569764026092275,14.609793838496916,1.565042260872111,175,13.228756555322953
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,130.71028066470666,4.271186609164921,393.5290419056911,8.434121881951622,BLINDED,BLINDED

name,extp,rel_err
AtoB,1.2988945836860144,0.030049017181030

In [5]:
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9",
#     "presel_noDetaJJ and M_jj > 600 and LT > 400 and ST > 1500 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
#     show_raw=True,
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,9.013904660083458,0.9561465942007488,2.3242513157526883,0.6191868265369841,BLINDED,BLINDED
presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,18.676956734430444,3.9892026320353797,0.0,0.0,9,3.0
presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,11.687424654201495,1.3361661060480508,3.4935710564344795,0.7378926693408709,BLINDED,BLINDED
presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,6.17581059009611,1.4513866924758507,112.02646875818817,4.532338703419458,BLINDED,BLINDED

name,extp,rel_err
AtoB,2.0720162281212233,0.2384791800994446
BtoC,0.6257670786727292,0.24226168310699114
CtoD,0.52

In [6]:
vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 2
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9",
#     "presel_noDetaJJ and M_jj > 600 and LT > 400 and ST > 900 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
#     show_raw=True,
    show_data=True
)

vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 0.5
print("")

vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 0.5
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9",
#     "presel_noDetaJJ and M_jj > 600 and LT > 400 and ST > 900 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
#     show_raw=True,
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,202.79078616421063,3.712187862108981,10.085911672694348,1.3074765499057894,148,12.165525060596439
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,278.2163093279004,6.053160403548995,0.8942036641677474,0.43743234037268064,200,14.142135623730951
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,229.29325258487154,4.881287544704813,14.609793838496916,1.565042260872111,175,13.228756555322953
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,154.9163623211776,4.931170793556183,393.5290419056911,8.434121881951622,BLINDED,BLINDED

name,extp,rel_err
AtoB,1.3719376239441847,0.02843342854841704